# Transfer Learning in Keras

In this notebook, we will implement transfer learning in Python using the pre-trained ResNet model. We will run two experiments - 1. **Freezing the base model weights**, adding a few layers to it at the end (fully connected etc.) and training the newly added layers, and 2. **Freezing the first 140 layers of ResNet** and retraining the rest.

Apart from this, you will learn **two important practical preprocessing techniques** in this notebook - **data augmentation** and **data generators**. The notebook is dividede into the following sections:
1. Importing libraries
2. Splitting into train and test set
3. Importing the pretrained ResNet model
4. Data Generators: Preprocessing and Generating Batch-Wise Data (On the Fly)
5. Training the Base Model (Using Batch-Wise Data Generation)
6. Freezing the initial-n layers and training the rest

For the dataset you have to create kaggle account and download from here

https://www.kaggle.com/alxmamaev/flowers-recognition

Below code was executed on cloud env so you can replace below line of code and read data directly using dataset you donwloaded

In [1]:
# For downloading the dataset
# Go to Kaggle.com -> My Account -> Create new API Token -> Upload kaggle.json in Jupyter Home
!mkdir ~/.kaggle
!cp ../kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n proxy -v http://10.150.0.3:3128
!kaggle datasets download -d alxmamaev/flowers-recognition
!unzip flowers-recognition.zip
!rm flowers-recognition.zip

- proxy is now set to: http://10.150.0.3:3128
100%|███████████████████████████████████████▊| 224M/225M [00:16<00:00, 14.9MB/s]
100%|████████████████████████████████████████| 225M/225M [00:16<00:00, 13.9MB/s]
Archive:  flowers-recognition.zip
   creating: flowers/
   creating: flowers/daisy/
  inflating: flowers/daisy/100080576_f52e8ee070_n.jpg  
  inflating: flowers/daisy/10140303196_b88d3d6cec.jpg  
  inflating: flowers/daisy/10172379554_b296050f82_n.jpg  
  inflating: flowers/daisy/10172567486_2748826a8b.jpg  
  inflating: flowers/daisy/10172636503_21bededa75_n.jpg  
  inflating: flowers/daisy/102841525_bd6628ae3c.jpg  
  inflating: flowers/daisy/10300722094_28fa978807_n.jpg  
  inflating: flowers/daisy/1031799732_e7f4008c03.jpg  
  inflating: flowers/daisy/10391248763_1d16681106_n.jpg  
  inflating: flowers/daisy/10437754174_22ec990b77_m.jpg  
  inflating: flowers/daisy/10437770546_8bb6f7bdd3_m.jpg  
  inflating: flowers/daisy/10437929963_bc13eebe0c.jpg  
  inflating: flowers/daisy/

  inflating: flowers/daisy/21402054779_759366efb0_n.jpg  
  inflating: flowers/daisy/21626652132_97e1318bb8_m.jpg  
  inflating: flowers/daisy/21652746_cc379e0eea_m.jpg  
  inflating: flowers/daisy/21805938544_bf6bb0e4bc.jpg  
  inflating: flowers/daisy/2213954589_c7da4b1486.jpg  
  inflating: flowers/daisy/22244161124_53e457bb66_n.jpg  
  inflating: flowers/daisy/22873310415_3a5674ec10_m.jpg  
  inflating: flowers/daisy/23095658544_7226386954_n.jpg  
  inflating: flowers/daisy/2331133004_582772d58f_m.jpg  
  inflating: flowers/daisy/2346726545_2ebce2b2a6.jpg  
  inflating: flowers/daisy/2349640101_212c275aa7.jpg  
  inflating: flowers/daisy/2365428551_39f83f10bf_n.jpg  
  inflating: flowers/daisy/2408024540_37f0be7cc0_n.jpg  
  inflating: flowers/daisy/2454280135_ac3aa75cdc_n.jpg  
  inflating: flowers/daisy/2454280137_e1637536ae_n.jpg  
  inflating: flowers/daisy/2473825306_62fd5f8785_n.jpg  
  inflating: flowers/daisy/2476937534_21b285aa46_n.jpg  
  inflating: flowers/daisy/24799564

  inflating: flowers/daisy/530738000_4df7e4786b.jpg  
  inflating: flowers/daisy/534547364_3f6b7279d2_n.jpg  
  inflating: flowers/daisy/538920244_59899a78f8_n.jpg  
  inflating: flowers/daisy/5434742166_35773eba57_m.jpg  
  inflating: flowers/daisy/5434901893_4550be3f84_m.jpg  
  inflating: flowers/daisy/5434913005_409c1e8b56_n.jpg  
  inflating: flowers/daisy/5434914569_e9b982fde0_n.jpg  
  inflating: flowers/daisy/5435513198_90ce39f1aa_n.jpg  
  inflating: flowers/daisy/5435521200_92029bbe2b_n.jpg  
  inflating: flowers/daisy/5435522104_1d6a61b431_n.jpg  
  inflating: flowers/daisy/54377391_15648e8d18.jpg  
  inflating: flowers/daisy/5459481183_18d2d49e44_m.jpg  
  inflating: flowers/daisy/5512287917_9f5d3f0f98_n.jpg  
  inflating: flowers/daisy/5547758_eea9edfd54_n.jpg  
  inflating: flowers/daisy/5561775629_a2b709b3a4_n.jpg  
  inflating: flowers/daisy/5574421625_61b1f49b3f_m.jpg  
  inflating: flowers/daisy/5577555349_2e8490259b.jpg  
  inflating: flowers/daisy/5586977262_6b24412

  inflating: flowers/dandelion/10200780773_c6051a7d71_n.jpg  
  inflating: flowers/dandelion/10294487385_92a0676c7d_m.jpg  
  inflating: flowers/dandelion/10437652486_aa86c14985.jpg  
  inflating: flowers/dandelion/10443973_aeb97513fc_m.jpg  
  inflating: flowers/dandelion/10477378514_9ffbcec4cf_m.jpg  
  inflating: flowers/dandelion/10486992895_20b344ce2d_n.jpg  
  inflating: flowers/dandelion/10617162044_8740d4dd9f_n.jpg  
  inflating: flowers/dandelion/10617191174_9a01753241_n.jpg  
  inflating: flowers/dandelion/10683189_bd6e371b97.jpg  
  inflating: flowers/dandelion/1074999133_1e4a1e042e.jpg  
  inflating: flowers/dandelion/10777398353_5a20bb218c.jpg  
  inflating: flowers/dandelion/10778387133_9141024b10.jpg  
  inflating: flowers/dandelion/10779476016_9130714dc0.jpg  
 extracting: flowers/dandelion/1080179756_5f05350a59.jpg  
  inflating: flowers/dandelion/10828951106_c3cd47983f.jpg  
  inflating: flowers/dandelion/10919961_0af657c4e8.jpg  
  inflating: flowers/dandelion/109468

  inflating: flowers/dandelion/17146644679_11aff3045c.jpg  
  inflating: flowers/dandelion/17147436650_c94ae24004_n.jpg  
  inflating: flowers/dandelion/17161833794_e1d92259d2_m.jpg  
  inflating: flowers/dandelion/17175932454_c052e205c1_n.jpg  
  inflating: flowers/dandelion/17189437699_a9171b6ae3.jpg  
  inflating: flowers/dandelion/17220096449_0e535989f0_n.jpg  
  inflating: flowers/dandelion/17243540220_65b98eb926_n.jpg  
  inflating: flowers/dandelion/17244252705_328e0bcda6.jpg  
  inflating: flowers/dandelion/17276354745_2e312a72b5_n.jpg  
  inflating: flowers/dandelion/17280886635_e384d91300_n.jpg  
  inflating: flowers/dandelion/17322195031_c2680809dc_m.jpg  
  inflating: flowers/dandelion/17344936845_fec4d626b7.jpg  
  inflating: flowers/dandelion/17346385582_7ba433dbbe.jpg  
  inflating: flowers/dandelion/17367866236_61abd4d243_n.jpg  
  inflating: flowers/dandelion/17388674711_6dca8a2e8b_n.jpg  
  inflating: flowers/dandelion/17388697431_0d84c427d1_n.jpg  
  inflating: flowe

...

  inflating: flowers/tulip/14084211971_0f921f11fe_n.jpg  
  inflating: flowers/tulip/14084749296_6143c74c72_m.jpg  
  inflating: flowers/tulip/14087326141_1906d5a373_n.jpg  
 extracting: flowers/tulip/14087361621_9fefb8dbef.jpg  
  inflating: flowers/tulip/14087425312_2b5846b570_n.jpg  
 extracting: flowers/tulip/14087439392_969444f56e.jpg  
  inflating: flowers/tulip/14087792403_f34f37ba3b_m.jpg  
  inflating: flowers/tulip/14087860553_bf4f8ec56d.jpg  
  inflating: flowers/tulip/14087892193_653a3ac7ca_n.jpg  
  inflating: flowers/tulip/14088017343_dd158d2eb5.jpg  
  inflating: flowers/tulip/14090534565_5857ce4b7c_n.jpg  
  inflating: flowers/tulip/14090546015_504c8becd1.jpg  
  inflating: flowers/tulip/14093565032_a8f1e349d1.jpg  
  inflating: flowers/tulip/14093884601_c87b5cd663_n.jpg  
  inflating: flowers/tulip/14093907931_dd8f642574.jpg  
  inflating: flowers/tulip/14094114202_4c1d7f1116.jpg  
  inflating: flowers/tulip/14094146241_0b9fa7b3e0.jpg  
  inflating: flowers/tulip/14097

  inflating: flowers/tulip/17969516345_d0081a5f18_n.jpg  
  inflating: flowers/tulip/17970201501_6ee7401206_n.jpg  
  inflating: flowers/tulip/17994129033_bbd0acba62_n.jpg  
  inflating: flowers/tulip/18245124970_e68fd3f3c3.jpg  
 extracting: flowers/tulip/18270448366_d5676dec64_z.jpg  
  inflating: flowers/tulip/18378581986_5cd1494d08_n.jpg  
  inflating: flowers/tulip/18378582936_ee7085c850.jpg  
  inflating: flowers/tulip/18406629611_4d1edcf23b_n.jpg  
  inflating: flowers/tulip/19268907364_d94a846fc8_n.jpg  
  inflating: flowers/tulip/19315333834_cfed7ae637_n.jpg  
  inflating: flowers/tulip/19317019453_8b24740faf_n.jpg  
  inflating: flowers/tulip/19324610724_1facd379ed_n.jpg  
  inflating: flowers/tulip/19413898445_69344f9956_n.jpg  
  inflating: flowers/tulip/19425920580_cdc8f49aed_n.jpg  
  inflating: flowers/tulip/19660257424_e9080d2ddc_n.jpg  
  inflating: flowers/tulip/19689668684_15089ae2fa_n.jpg  
  inflating: flowers/tulip/19689681344_b05ac361f2_n.jpg  
  inflating: flowe

 extracting: flowers/tulip/3254533919_cb0b8af26c.jpg  
  inflating: flowers/tulip/3282751630_45c2665034_m.jpg  
  inflating: flowers/tulip/3396033831_bb88d93630.jpg  
  inflating: flowers/tulip/3404038663_f62cf8eba3_n.jpg  
  inflating: flowers/tulip/3421027755_cdb8fef8e8_n.jpg  
  inflating: flowers/tulip/3422915985_9bf7264d36.jpg  
 extracting: flowers/tulip/3430229687_32645b5738.jpg  
  inflating: flowers/tulip/3433265727_0b8022e091.jpg  
  inflating: flowers/tulip/3446285408_4be9c0fded_m.jpg  
  inflating: flowers/tulip/3449794006_8c289840aa.jpg  
  inflating: flowers/tulip/3454461550_64d6e726bf_m.jpg  
  inflating: flowers/tulip/3455026124_d66cafb9fc.jpg  
  inflating: flowers/tulip/3457017604_90e4de7480_m.jpg  
  inflating: flowers/tulip/3459922572_bc8516b5fe_m.jpg  
  inflating: flowers/tulip/3474066174_8d3b3e8f97_n.jpg  
  inflating: flowers/tulip/3476945045_97ff41e8ec_n.jpg  
 extracting: flowers/tulip/3485767306_6db7bdf536.jpg  
  inflating: flowers/tulip/3498663243_42b39b418

  inflating: flowers/tulip/5674127693_1ddbd81097.jpg  
  inflating: flowers/tulip/5674132053_b40a7d32ca.jpg  
  inflating: flowers/tulip/5674134129_2db5136cba.jpg  
  inflating: flowers/tulip/5674167473_ac696c8989_n.jpg  
  inflating: flowers/tulip/5674170543_73e3f403fb.jpg  
  inflating: flowers/tulip/5674695558_61397a1584.jpg  
  inflating: flowers/tulip/5674704952_9bd225ed9e_n.jpg  
  inflating: flowers/tulip/5674707464_dc18de05b1.jpg  
  inflating: flowers/tulip/5680695867_baff72fc7c.jpg  
 extracting: flowers/tulip/5682463466_d3e641cb8b.jpg  
 extracting: flowers/tulip/5687705933_55a8c2dbac.jpg  
  inflating: flowers/tulip/5691090657_2f1e9bf49e_n.jpg  
  inflating: flowers/tulip/5691100579_4a2767360a.jpg  
  inflating: flowers/tulip/5691672942_70a93d70fc.jpg  
  inflating: flowers/tulip/5697471591_200ff951fa_n.jpg  
  inflating: flowers/tulip/5698944116_fd35fe6bea.jpg  
  inflating: flowers/tulip/5700394524_dc6f8fa9cd_n.jpg  
  inflating: flowers/tulip/5700466891_2bcb17fa68_n.jpg 

  inflating: flowers/tulip/8585102511_fc452e6700_n.jpg  
  inflating: flowers/tulip/8585102913_d80d3e2ff7_n.jpg  
  inflating: flowers/tulip/8585103457_d64697c3cf_n.jpg  
  inflating: flowers/tulip/8586204750_2891bd2ec9_n.jpg  
  inflating: flowers/tulip/8586205168_8294e67195_n.jpg  
  inflating: flowers/tulip/8586205446_8953a6c70e_n.jpg  
  inflating: flowers/tulip/8586205946_cda045f3f8_n.jpg  
  inflating: flowers/tulip/8601596054_33e40c2a7a.jpg  
  inflating: flowers/tulip/8603340662_0779bd87fd.jpg  
  inflating: flowers/tulip/8605564823_7a59d3d92a.jpg  
  inflating: flowers/tulip/8614237582_74417799f4_m.jpg  
  inflating: flowers/tulip/8619064872_dea79a9eb9.jpg  
  inflating: flowers/tulip/8622237974_b362574785_n.jpg  
  inflating: flowers/tulip/8623170936_83f4152431.jpg  
  inflating: flowers/tulip/8623173256_3f0eb4c506.jpg  
  inflating: flowers/tulip/8628453641_6f87755815_m.jpg  
  inflating: flowers/tulip/8659691170_09db83d023.jpg  
  inflating: flowers/tulip/8668973377_c69527d

### Importing Libraries

In [23]:
# !pip install keras

In [24]:
# import libraries
import os
import shutil
import numpy as np
import glob   
import keras.backend as K
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D
from keras.models import Model, load_model
from keras import optimizers
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
import scipy.misc
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

# for reading images
from matplotlib.pyplot import imshow
%matplotlib inline

# channels last is the format used by tensorflow 
K.set_image_data_format('channels_last')
K.set_learning_phase(1)


### Splitting Into Train and Test

Let's now split the data into train and test directories. Firstly, note that the most common way to organize (images) data is to create two/three directories - train and test (or validation) having n-subdirectories, each subdirectory being a class (here, five subdirs for the five flower types).

The following function creates two directories - train and test, each having five subdirectories (sunflower, dandelion, rose, tulip, daisy).

In [25]:
# set to where the 'flowers' directory is located
data_dir = 'flowers'

# Training data dir
training_dir = 'Train'

# Test data dir
testing_dir = 'Test'

# Ratio of training and testing data
train_test_ratio = 0.8 


def split_dataset_into_test_and_train_sets(all_data_dir = data_dir, training_data_dir = training_dir, testing_data_dir=testing_dir, train_test_ratio = 0.8):

    # recreate test and train directories if they don't exist
    if not os.path.exists(training_data_dir):
        os.mkdir(training_data_dir)

    if not os.path.exists(testing_data_dir):
        os.mkdir(testing_data_dir)               
    
    num_training_files = 0
    num_testing_files = 0

    # iterate through the data directory 
    for subdir, dirs, files in os.walk(all_data_dir):
        
        category_name = os.path.basename(subdir)

        if category_name == os.path.basename(all_data_dir):
            continue

        training_data_category_dir = training_data_dir + '/' + category_name
        testing_data_category_dir = testing_data_dir + '/' + category_name
        
        # creating subdirectory for each sub category
        if not os.path.exists(training_data_category_dir):
            os.mkdir(training_data_category_dir)   

        if not os.path.exists(testing_data_category_dir):
            os.mkdir(testing_data_category_dir)
            
        file_list = glob.glob(subdir + '/*.jpg')

        print(str(category_name) + ' has ' + str(len(files)) + ' images') 
        random_set = np.random.permutation((file_list))
        
        # copy percentage of data from each category to train and test directory
        train_list = random_set[:round(len(random_set)*(train_test_ratio))] 
        test_list = random_set[-round(len(random_set)*(1-train_test_ratio)):]
        
        for lists in train_list : 
            shutil.copy(lists, training_data_dir + '/' + category_name + '/' )
            num_training_files += 1
  
        for lists in test_list : 
            shutil.copy(lists, testing_data_dir + '/' + category_name + '/' )
            num_testing_files += 1
  

    print("Processed " + str(num_training_files) + " training files.")
    print("Processed " + str(num_testing_files) + " testing files.")

In [30]:
# split into train and test directories
split_dataset_into_test_and_train_sets()

sunflower has 734 images
dandelion has 1055 images
rose has 784 images
tulip has 984 images
daisy has 769 images
Processed 3458 training files.
Processed 865 testing files.


### Importing the Pre-Trained Model 

Let's now import the pretrained ResNet model. In the first experiment, we will use the pretrained weights (from Imagenet) of ResNet. The argument `include_top = False` specifies that we do not want to import the top layers (the last ones, which are typically pooling, FC, softmax etc.). We'll add some of our own last layers (a global average poooling layer and a final softmax) and train just those.

In [31]:
# number of classes 
num_classes = 5

def get_model():
    
    # Get base model: ResNet50 
    base_model = ResNet50(weights='imagenet', include_top=False)
    
    # freeze the layers in base model
    for layer in base_model.layers:
        layer.trainable = False
        
    # Get the output from the base model 
    base_model_ouput = base_model.output
    
    # Adding our own layers at the end
    # global average pooling: computes the average of all values in the feature map
    x = GlobalAveragePooling2D()(base_model_ouput)
    
    # fully connected and 5-softmax layer
    x = Dense(512, activation='relu')(x)
    x = Dense(num_classes, activation='softmax', name='fcnew')(x)
    
    model = Model(inputs=base_model.input, outputs=x)
    return model

In [32]:
# Get the model
model = get_model()

# compile it
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# summary of model
model.summary()

/usr/local/lib/python3.5/dist-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

Notice that the total number of parameters in the model is about 24 million, though the number of trainable parameters is only about 1 million.

Let's now see how we'll feed the data to the model.

### Data Generators: Preprocessing and Generating Batch-Wise Data (On the Fly)

We will now implement an incredibly useful preprocessing technique - **data augmentation** using **data generators**.

You will learn preprocessing techniques in detail in the next industry session, though they're quire easy to understand anyway. Here's a quick overview.

**Data Augmentation** is a commonly used technique in image processing used to 'create' more training data. It basically modifies the original training images a bit (e.g. rotates them by a few degrees, changes the colour shades a little, etc.) to 'create' (or augment) new training images. The basic reason to do this is to **increase the amount of variance** in the training data. It is possible to do this with images because if you rotate the image of (say) a dog (or change the colours a bit, stretch the image horizontally etc.), it stays a dog. Thus, you can create many images from each training image while the label stays the same.

In the code below, we have specified the augmentation techniques as `shear_range=0.2, zoom_range=0.2, horizontal_flip=True`. Shear 'stretches' the images, zoom_range zooms them in, and horizontal_flip 'flips' them around horizontally.

Now, in the code below, you will notice that we have something called `ImageDataGenerator` - lets understand what it does.

**Data generators** are used to **feed data points in batches** to the model. The main reason to use them is that they are efficient (compared to feeding one data point at a time, or all of them at once which will require a lot of memory). What's cooler about them is that they (at least in keras) can preprocess the images and create augmented ones *on the fly*, i.e. as the batches are fed to the model, the augmented images are created and preprocessed. This eliminates the need to store the augmented images separately.

An important point to note is that you **never augment the validation or test sets**, only the training set. This is because test (and validation) sets are supposed to be representative of the real images you'll get in the real world. However, you do apply basic preprocessing to validation and test sets (scaling/centering them etc.).

In the code below, the method `flow_from_directory` 'flows' the data in batches from the training and test directories. It is an instance of `ImageDataGenerator` where we specify the preprocessing and augmentation techniques that we want to use. In this case, we are just using the standard preprocessing techniques that come with the `preprocess_input` module in keras.

You can <a href="https://keras.io/preprocessing/image/">read about data generators here</a>.

In [35]:
# Using ImageDataGenerator for pre-processing

image_size = 224
batch_size = 64

# help(ImageDataGenerator)
train_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input, 
                                    shear_range=0.2, zoom_range=0.2, 
                                    horizontal_flip=True)

# do only basic preprocessing for validation data (no data augmentation)
valid_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)

# create data generator objects
train_generator = train_data_gen.flow_from_directory(training_dir, (image_size,image_size), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_data_gen.flow_from_directory(testing_dir, (image_size,image_size), batch_size=batch_size, class_mode='categorical')

Found 4145 images belonging to 5 classes.
Found 1552 images belonging to 5 classes.


Note that the number of training and validation images is increased (because of data augmentation).

### Training the Base Model (Using Batch-Wise Data Generation)

Let's now train the model. When we use data generators, we use the `model.fit_generator` method rather than the usual `model.fit`.

In [36]:
# Training the newly added layers 
epochs = 5

# flow data (in batches) from directories (while simultaneously preprocessing/augmenting
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=epochs,
    verbose=1)

Epoch 1/5
64/64 [==============================] - 76s 1s/step - loss: 0.8122 - acc: 0.7107 - val_loss: 0.4851 - val_acc: 0.8460
Epoch 2/5
64/64 [==============================] - 70s 1s/step - loss: 0.4480 - acc: 0.8575 - val_loss: 0.3669 - val_acc: 0.8756
Epoch 3/5
64/64 [==============================] - 70s 1s/step - loss: 0.3655 - acc: 0.8809 - val_loss: 0.3189 - val_acc: 0.8924
Epoch 4/5
64/64 [==============================] - 70s 1s/step - loss: 0.3323 - acc: 0.8887 - val_loss: 0.2842 - val_acc: 0.9059
Epoch 5/5
64/64 [==============================] - 69s 1s/step - loss: 0.2912 - acc: 0.8986 - val_loss: 0.2548 - val_acc: 0.9162


### Freezing the Initial-n Layers and Training the Rest

Let's now try another variant of transfer learning. We will freeze the first 140 layers of ResNet (with the hypopthesis that they have learnt to extract some useful generic features from their ImageNet experience) and train the rest of the layers.

In [37]:
epochs = 10

# training the model after 140 layers
split_at = 140
for layer in model.layers[:split_at]: layer.trainable = False
for layer in model.layers[split_at:]: layer.trainable = True
    
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Choosing lower learning rate for fine-tuning
# learning rate is generally 10-1000 times lower than normal learning rate when we are fine tuning the initial layers
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=epochs,
    verbose=1)

Epoch 1/10
64/64 [==============================] - 78s 1s/step - loss: 0.2571 - acc: 0.9122 - val_loss: 0.2024 - val_acc: 0.9330
Epoch 2/10
64/64 [==============================] - 71s 1s/step - loss: 0.1992 - acc: 0.9319 - val_loss: 0.1569 - val_acc: 0.9504
Epoch 3/10
64/64 [==============================] - 73s 1s/step - loss: 0.1715 - acc: 0.9461 - val_loss: 0.1294 - val_acc: 0.9652
Epoch 4/10
64/64 [==============================] - 71s 1s/step - loss: 0.1407 - acc: 0.9526 - val_loss: 0.1064 - val_acc: 0.9652
Epoch 5/10
64/64 [==============================] - 71s 1s/step - loss: 0.1123 - acc: 0.9670 - val_loss: 0.0926 - val_acc: 0.9768
Epoch 6/10
64/64 [==============================] - 69s 1s/step - loss: 0.0913 - acc: 0.9753 - val_loss: 0.0826 - val_acc: 0.9762
Epoch 7/10
64/64 [==============================] - 69s 1s/step - loss: 0.0804 - acc: 0.9801 - val_loss: 0.0769 - val_acc: 0.9768
Epoch 8/10
64/64 [==============================] - 66s 1s/step - loss: 0.0664 - acc: 0.98

In [27]:
print('Training complete')

Training complete
